In [1]:
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("../src/")
%pwd

'/home/jovyan/nfs-home/scalable_ml_pipelines/src'

In [3]:
import socket

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [4]:
name_space = "eabraham-373705"

# Master node
kubernetes_master_url = "k8s://https://10.32.7.103:6443"

# Resource settings
driver_cores = "6"
executor_cores = "6"
driver_memory = "22g"
executor_memory = "22g"
executor_memory_overhead = "2g"

# These are the limits
cpu_limit = "3"  # 12 cores
memory_limit = "32g"  # Upto 32 GB
executor_limit = "8"

In [5]:
from pyspark.sql import SparkSession

APP_NAME = 'scalables_executor'


spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(kubernetes_master_url)\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", executor_cores)\
    .config("spark.executor.memory", executor_memory)\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.2")\
    .config("spark.kubernetes.executor.limit.cores", executor_limit)\
    .config("spark.kubernetes.namespace", name_space)\
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.deleteOnTermination", "false") \
    .config("spark.kubernetes.container.image.pullPolicy", "Always") \
    .config("spark.kubernetes.container.image", "node03.st:5000/pyspark-hdfs-jupyter:eabraham-373705-v4-executor")\
    .config("spark.local.dir", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .getOrCreate()

23/12/29 09:09:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 09:09:08 WARN spark.SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/12/29 09:09:09 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
spark

In [7]:
from us_used_cars_ml_pipeline.pipelines.new_data_pipeline import run

[2023-12-29 09:09:16,886: 145: numexpr.utils: INFO: utils:  Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2023-12-29 09:09:16,888: 157: numexpr.utils: INFO: utils:  NumExpr defaulting to 8 threads.]


In [8]:
%%time
run(spark, do_conversion=False, calculate_metrics=True)

[2023-12-29 09:09:17,735: 18: us_used_cars_ml_pipeline_logger: INFO: new_data_pipeline:  === STARTING PROCESSING NEW DATA ===]
[2023-12-29 09:09:17,753: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/config/config.yaml loaded successfully]
[2023-12-29 09:09:17,756: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/params.yaml loaded successfully]
[2023-12-29 09:09:17,759: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/schema.yaml loaded successfully]
[2023-12-29 09:09:17,761: 29: us_used_cars_ml_pipeline_logger: INFO: new_data_pipeline:  === I. COMPONENTS HAVE BEEN INITIALIZED ===]
[2023-12-29 09:09:17,762: 36: us_used_cars_ml_pipeline_logger: INFO: new_data_pipeline:  === II. DATA CONVERSION STAGE HAS BEEN SKIPPED ===]


[2023-12-29 09:09:21,763: 848: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  Raw data has been read]
[2023-12-29 09:09:22,402: 856: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  PART I. STARTING]
[2023-12-29 09:09:22,415: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/data_types.yaml loaded successfully]
[2023-12-29 09:09:22,424: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_with_nans.yaml loaded successfully]
[2023-12-29 09:09:22,428: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/popular_options.yaml loaded successfully]
[2023-12-29 09:09:22,434: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_to_encode.yaml loaded successfully]
[2023-12-29 09:09:22,559: 644: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  1. Rows with nans in specified colu

  0%|          | 0/43 [00:00<?, ?it/s]

[2023-12-29 09:09:23,401: 669: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  3. Values in specified columns have been converted]
[2023-12-29 09:09:23,593: 674: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  4. Features power and torque have been splitted]
[2023-12-29 09:09:24,126: 678: us_used_cars_ml_pipeline_logger: INFO: data_cleaning:  5. Popular options features have been extracted]


  0%|          | 0/11 [00:00<?, ?it/s]

...

[2023-12-29 09:11:44,865: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/importances/LinearRegression.yaml loaded successfully]
[2023-12-29 09:11:45,853: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/importances/RandomForestRegressor.yaml loaded successfully]
[2023-12-29 09:11:47,236: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/importances/GBTRegressor.yaml loaded successfully]


23/12/29 09:11:47 WARN execution.CacheManager: Asked to cache already cached data.


[2023-12-29 09:11:48,415: 213: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  First level predictions have been calculated]


  0%|          | 0/3 [00:00<?, ?it/s]

[2023-12-29 09:11:49,053: 217: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  First level predictions have been merged]
[2023-12-29 09:11:49,644: 224: us_used_cars_ml_pipeline_logger: INFO: stacking_regressor_modeling:  Second level prediction has been calculated]
[2023-12-29 09:11:49,645: 49: us_used_cars_ml_pipeline_logger: INFO: new_data_pipeline:  === V. PREDICTIONS HAVE BEEN CALCULATED AND SAVED===]
[2023-12-29 09:11:49,724: 149: us_used_cars_ml_pipeline_logger: INFO: metrics_calculation:  Raw data has been loaded]
[2023-12-29 09:11:49,800: 153: us_used_cars_ml_pipeline_logger: INFO: metrics_calculation:  Predictions have been loaded]
[2023-12-29 09:11:51,727: 157: us_used_cars_ml_pipeline_logger: INFO: metrics_calculation:  Metrics have been calculated and saved]
[2023-12-29 09:11:51,729: 54: us_used_cars_ml_pipeline_logger: INFO: new_data_pipeline:  === VI. METRICS CALCULATION STAGE HAS BEEN COMPLETED ===]
[2023-12-29 09:11:51,730: 58: us_used_cars_ml_pipel

# Checking predictions and metrics

In [9]:
df = spark.read.json('hdfs:///home/eabraham-373705/data/metrics/NEW_metrics.json')
df.show()

+------+---------+
|metric|    value|
+------+---------+
|  rmse|  5501.19|
|   mae| 4349.733|
|    r2|0.8691952|
+------+---------+



In [20]:
df = spark.read.parquet('/home/eabraham-373705/data/predictions/NEW_second_level_predictions.parquet', 
                        header=True, inferSchema=True)
df2 = spark.read.parquet('/home/eabraham-373705/data/raw/NEW_raw_data.parquet', 
                        header=True, inferSchema=True)
df2 = df2.select('vin', 'price')
df = df.join(df2, on='vin', how='left')

In [22]:
df.show(10)

+-----------------+------------------+-------+
|              vin|        prediction|  price|
+-----------------+------------------+-------+
|1G11C5SA9DF208064| 9833.636111618302| 8288.0|
|KMHL34JJXLA012167| 23463.89605507132|31600.0|
|1FTEW1CP9LKE93480| 43493.38756785468|34315.0|
|2GNFLHEK5E6202302| 12171.82698219005|10617.0|
|5UXKR6C50J0U15182| 59107.40822059929|54900.0|
|2T3BFREV7HW637734|16158.662423488715|21250.0|
|5TDKK3DCXBS068413|10403.470680028315| 7900.0|
|1GYS4HKJ4LR117429|48805.301917833756|61999.0|
|2C4RC1BG6KR597128|20852.951868415043|21081.0|
|2FMPK4G97JBC39531|28428.957823514847|23449.0|
+-----------------+------------------+-------+
only showing top 10 rows

